In [1]:
import requests
import json
import prettytable
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2 
import io

In [2]:
pd.set_option('display.max_columns', None)

# 2018

In [3]:
file_2018 = "data_files/bls_metro_2018_clean.csv"
bls_metro_2018_df = pd.read_csv(file_2018,index_col=None)
bls_metro_2018_df.head(5)

,Unnamed: 0,prim_state,area,area_name,occ_code,occ_title,occ_group,tot_emp,emp_prse,jobs_1000,loc quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,Date
0,0,AK,11260,"Anchorage, AK",00-0000,All Occupations,total,171770.0,1.1,1000.000,1.00,28.57,59430.0,2.1,11.41,14.94,22.68,35.82,51.21,23740.0,31070.0,47170.0,74510.0,106510.0,May 2018
1,1,AK,11260,"Anchorage, AK",11-0000,Management Occupations,major,10610.0,2.5,61.779,1.17,55.59,115630.0,1.8,26.68,36.03,48.94,64.70,93.24,55490.0,74950.0,101800.0,134580.0,193930.0,May 2018
2,5,AK,11260,"Anchorage, AK",11-2011,Advertising and Promotions Managers,detailed,50.0,16.5,0.282,1.62,45.98,95640.0,7.8,23.96,32.01,37.84,49.67,89.90,49830.0,66580.0,78710.0,103320.0,186990.0,May 2018
3,6,AK,11260,"Anchorage, AK",11-2021,Marketing Managers,detailed,150.0,12.9,0.867,0.52,49.62,103220.0,3.4,28.73,35.57,45.62,59.58,77.63,59760.0,73980.0,94890.0,123930.0,161480.0,May 2018
4,8,AK,11260,"Anchorage, AK",11-2031,Public Relations and Fundraising Managers,detailed,100.0,13.7,0.557,1.11,55.12,114650.0,8.4,28.48,35.40,48.55,65.02,78.77,59240.0,73640.0,100980.0,135240.0,163840.0,May 2018


In [4]:
bls_metro_2018_df.drop(['Unnamed: 0','occ_group'] , axis=1, inplace=True)

bls_metro_2018_df.head(5)

,prim_state,area,area_name,occ_code,occ_title,tot_emp,emp_prse,jobs_1000,loc quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,Date
0,AK,11260,"Anchorage, AK",00-0000,All Occupations,171770.0,1.1,1000.000,1.00,28.57,59430.0,2.1,11.41,14.94,22.68,35.82,51.21,23740.0,31070.0,47170.0,74510.0,106510.0,May 2018
1,AK,11260,"Anchorage, AK",11-0000,Management Occupations,10610.0,2.5,61.779,1.17,55.59,115630.0,1.8,26.68,36.03,48.94,64.70,93.24,55490.0,74950.0,101800.0,134580.0,193930.0,May 2018
2,AK,11260,"Anchorage, AK",11-2011,Advertising and Promotions Managers,50.0,16.5,0.282,1.62,45.98,95640.0,7.8,23.96,32.01,37.84,49.67,89.90,49830.0,66580.0,78710.0,103320.0,186990.0,May 2018
3,AK,11260,"Anchorage, AK",11-2021,Marketing Managers,150.0,12.9,0.867,0.52,49.62,103220.0,3.4,28.73,35.57,45.62,59.58,77.63,59760.0,73980.0,94890.0,123930.0,161480.0,May 2018
4,AK,11260,"Anchorage, AK",11-2031,Public Relations and Fundraising Managers,100.0,13.7,0.557,1.11,55.12,114650.0,8.4,28.48,35.40,48.55,65.02,78.77,59240.0,73640.0,100980.0,135240.0,163840.0,May 2018


In [5]:
bls_metro_2018_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118961 entries, 0 to 118960
Data columns (total 23 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   prim_state    118961 non-null  object 
 1   area          118961 non-null  int64  
 2   area_name     118961 non-null  object 
 3   occ_code      118961 non-null  object 
 4   occ_title     118961 non-null  object 
 5   tot_emp       118961 non-null  float64
 6   emp_prse      118961 non-null  float64
 7   jobs_1000     118961 non-null  float64
 8   loc quotient  118961 non-null  float64
 9   h_mean        118961 non-null  float64
 10  a_mean        118961 non-null  float64
 11  mean_prse     118961 non-null  float64
 12  h_pct10       118961 non-null  float64
 13  h_pct25       118961 non-null  float64
 14  h_median      118961 non-null  float64
 15  h_pct75       118961 non-null  float64
 16  h_pct90       118961 non-null  float64
 17  a_pct10       118961 non-null  float64
 18  a_pc

In [6]:
# remove # * from dataframe 
# bls_metro_2018_df=bls_metro_2018_df.replace('\*','',regex=True)
# bls_metro_2018_df

In [7]:
# convert column name area_name to city_state

bls_metro_2018_df.rename(columns = {'area_name':'citi_state','loc quotient':'loc_quotient'}, inplace = True)


In [8]:
bls_metro_2018_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118961 entries, 0 to 118960
Data columns (total 23 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   prim_state    118961 non-null  object 
 1   area          118961 non-null  int64  
 2   citi_state    118961 non-null  object 
 3   occ_code      118961 non-null  object 
 4   occ_title     118961 non-null  object 
 5   tot_emp       118961 non-null  float64
 6   emp_prse      118961 non-null  float64
 7   jobs_1000     118961 non-null  float64
 8   loc_quotient  118961 non-null  float64
 9   h_mean        118961 non-null  float64
 10  a_mean        118961 non-null  float64
 11  mean_prse     118961 non-null  float64
 12  h_pct10       118961 non-null  float64
 13  h_pct25       118961 non-null  float64
 14  h_median      118961 non-null  float64
 15  h_pct75       118961 non-null  float64
 16  h_pct90       118961 non-null  float64
 17  a_pct10       118961 non-null  float64
 18  a_pc

# 2019

In [9]:
file_2019 = "data_files/bls_metro_2019_clean.csv"
bls_metro_2019_df= pd.read_csv(file_2019,index_col=None)
bls_metro_2019_df.head()

,Unnamed: 0,area,area_title,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
0,0,10180,"Abilene, TX",00-0000,All Occupations,total,66510.0,3.1,1000.000,1.00,20.08,41760.0,1.8,8.94,10.97,15.64,24.05,35.21,18580.0,22810.0,32530.0,50030.0,73230.0
1,1,10180,"Abilene, TX",11-0000,Management Occupations,major,2670.0,5.8,40.137,0.73,43.51,90500.0,2.2,17.86,27.22,37.51,51.88,75.57,37140.0,56610.0,78030.0,107910.0,157190.0
2,2,10180,"Abilene, TX",11-1021,General and Operations Managers,detailed,1100.0,7.7,16.499,1.01,40.39,84000.0,3.7,10.97,24.21,35.51,51.38,74.29,22810.0,50350.0,73850.0,106870.0,154520.0
3,3,10180,"Abilene, TX",11-2022,Sales Managers,detailed,110.0,18.8,1.728,0.63,59.96,124710.0,6.6,22.77,34.69,57.65,74.55,99.80,47360.0,72140.0,119920.0,155050.0,207570.0
4,5,10180,"Abilene, TX",11-3010,Administrative Services and Facilities Managers,detailed,150.0,12.3,2.197,1.05,40.55,84350.0,6.7,25.17,28.25,34.53,43.93,57.93,52360.0,58760.0,71810.0,91380.0,120490.0


In [10]:
bls_metro_2019_df.drop(['Unnamed: 0','o_group'] , axis=1, inplace=True)

bls_metro_2019_df.head(5)

,area,area_title,occ_code,occ_title,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
0,10180,"Abilene, TX",00-0000,All Occupations,66510.0,3.1,1000.000,1.00,20.08,41760.0,1.8,8.94,10.97,15.64,24.05,35.21,18580.0,22810.0,32530.0,50030.0,73230.0
1,10180,"Abilene, TX",11-0000,Management Occupations,2670.0,5.8,40.137,0.73,43.51,90500.0,2.2,17.86,27.22,37.51,51.88,75.57,37140.0,56610.0,78030.0,107910.0,157190.0
2,10180,"Abilene, TX",11-1021,General and Operations Managers,1100.0,7.7,16.499,1.01,40.39,84000.0,3.7,10.97,24.21,35.51,51.38,74.29,22810.0,50350.0,73850.0,106870.0,154520.0
3,10180,"Abilene, TX",11-2022,Sales Managers,110.0,18.8,1.728,0.63,59.96,124710.0,6.6,22.77,34.69,57.65,74.55,99.80,47360.0,72140.0,119920.0,155050.0,207570.0
4,10180,"Abilene, TX",11-3010,Administrative Services and Facilities Managers,150.0,12.3,2.197,1.05,40.55,84350.0,6.7,25.17,28.25,34.53,43.93,57.93,52360.0,58760.0,71810.0,91380.0,120490.0


In [11]:
bls_metro_2019_df["Date"]="May 2019"

In [12]:
bls_metro_2019_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118165 entries, 0 to 118164
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   area          118165 non-null  int64  
 1   area_title    118165 non-null  object 
 2   occ_code      118165 non-null  object 
 3   occ_title     118165 non-null  object 
 4   tot_emp       118165 non-null  float64
 5   emp_prse      118165 non-null  float64
 6   jobs_1000     118165 non-null  float64
 7   loc_quotient  118165 non-null  float64
 8   h_mean        118165 non-null  float64
 9   a_mean        118165 non-null  float64
 10  mean_prse     118165 non-null  float64
 11  h_pct10       118165 non-null  float64
 12  h_pct25       118165 non-null  float64
 13  h_median      118165 non-null  float64
 14  h_pct75       118165 non-null  float64
 15  h_pct90       118165 non-null  float64
 16  a_pct10       118165 non-null  float64
 17  a_pct25       118165 non-null  float64
 18  a_me

In [13]:
bls_metro_2019_df.rename(columns = {'area_title':'citi_state'}, inplace = True)

In [14]:
bls_metro_2019_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118165 entries, 0 to 118164
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   area          118165 non-null  int64  
 1   citi_state    118165 non-null  object 
 2   occ_code      118165 non-null  object 
 3   occ_title     118165 non-null  object 
 4   tot_emp       118165 non-null  float64
 5   emp_prse      118165 non-null  float64
 6   jobs_1000     118165 non-null  float64
 7   loc_quotient  118165 non-null  float64
 8   h_mean        118165 non-null  float64
 9   a_mean        118165 non-null  float64
 10  mean_prse     118165 non-null  float64
 11  h_pct10       118165 non-null  float64
 12  h_pct25       118165 non-null  float64
 13  h_median      118165 non-null  float64
 14  h_pct75       118165 non-null  float64
 15  h_pct90       118165 non-null  float64
 16  a_pct10       118165 non-null  float64
 17  a_pct25       118165 non-null  float64
 18  a_me

# 2020

In [15]:
file_2020 = "data_files/bls_50metro_2020_clean.csv"
bls_metro_2020_df= pd.read_csv(file_2020)
bls_metro_2020_df.head(5)

,Unnamed: 0,area,area_title,prim_state,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
0,0,10180,"Abilene, TX",TX,00-0000,All Occupations,total,66060.0,1.9,1000.000,1.00,20.64,42930.0,1.8,9.13,11.40,16.41,24.82,36.23,18990.0,23710.0,34130.0,51620.0,75370.0
1,1,10180,"Abilene, TX",TX,11-0000,Management Occupations,major,2910.0,4.5,44.068,0.77,42.87,89160.0,2.2,17.38,26.74,36.37,51.68,73.88,36150.0,55620.0,75640.0,107500.0,153670.0
2,2,10180,"Abilene, TX",TX,11-1021,General and Operations Managers,detailed,1320.0,7.4,20.036,1.19,40.38,83990.0,3.5,12.67,24.02,32.96,50.11,72.93,26350.0,49960.0,68550.0,104230.0,151700.0
3,4,10180,"Abilene, TX",TX,11-2030,Public Relations and Fundraising Managers,detailed,40.0,31.5,0.549,0.94,45.93,95540.0,20.7,28.71,32.78,36.24,41.23,83.81,59710.0,68180.0,75380.0,85760.0,174320.0
4,5,10180,"Abilene, TX",TX,11-3010,Administrative Services and Facilities Managers,detailed,140.0,12.3,2.045,0.92,37.79,78600.0,3.5,24.93,28.28,35.06,45.45,55.25,51850.0,58820.0,72920.0,94530.0,114920.0


In [16]:
bls_metro_2020_df.drop(['Unnamed: 0','o_group'] , axis=1, inplace=True)

bls_metro_2020_df.head(5)

,area,area_title,prim_state,occ_code,occ_title,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
0,10180,"Abilene, TX",TX,00-0000,All Occupations,66060.0,1.9,1000.000,1.00,20.64,42930.0,1.8,9.13,11.40,16.41,24.82,36.23,18990.0,23710.0,34130.0,51620.0,75370.0
1,10180,"Abilene, TX",TX,11-0000,Management Occupations,2910.0,4.5,44.068,0.77,42.87,89160.0,2.2,17.38,26.74,36.37,51.68,73.88,36150.0,55620.0,75640.0,107500.0,153670.0
2,10180,"Abilene, TX",TX,11-1021,General and Operations Managers,1320.0,7.4,20.036,1.19,40.38,83990.0,3.5,12.67,24.02,32.96,50.11,72.93,26350.0,49960.0,68550.0,104230.0,151700.0
3,10180,"Abilene, TX",TX,11-2030,Public Relations and Fundraising Managers,40.0,31.5,0.549,0.94,45.93,95540.0,20.7,28.71,32.78,36.24,41.23,83.81,59710.0,68180.0,75380.0,85760.0,174320.0
4,10180,"Abilene, TX",TX,11-3010,Administrative Services and Facilities Managers,140.0,12.3,2.045,0.92,37.79,78600.0,3.5,24.93,28.28,35.06,45.45,55.25,51850.0,58820.0,72920.0,94530.0,114920.0


In [17]:
bls_metro_2020_df["Date"]="May 2020"

In [18]:
bls_metro_2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115629 entries, 0 to 115628
Data columns (total 23 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   area          115629 non-null  int64  
 1   area_title    115629 non-null  object 
 2   prim_state    115629 non-null  object 
 3   occ_code      115629 non-null  object 
 4   occ_title     115629 non-null  object 
 5   tot_emp       115629 non-null  float64
 6   emp_prse      115629 non-null  float64
 7   jobs_1000     115629 non-null  float64
 8   loc_quotient  115629 non-null  float64
 9   h_mean        115629 non-null  float64
 10  a_mean        115629 non-null  float64
 11  mean_prse     115629 non-null  float64
 12  h_pct10       115629 non-null  float64
 13  h_pct25       115629 non-null  float64
 14  h_median      115629 non-null  float64
 15  h_pct75       115629 non-null  float64
 16  h_pct90       115629 non-null  float64
 17  a_pct10       115629 non-null  float64
 18  a_pc

In [19]:
# Match columns in 3 df 2018,2019,2020
bls_metro_2020_df.rename(columns = {'area_title':'citi_state'}, inplace = True)

In [20]:
#drop columns 
bls_metro_2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115629 entries, 0 to 115628
Data columns (total 23 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   area          115629 non-null  int64  
 1   citi_state    115629 non-null  object 
 2   prim_state    115629 non-null  object 
 3   occ_code      115629 non-null  object 
 4   occ_title     115629 non-null  object 
 5   tot_emp       115629 non-null  float64
 6   emp_prse      115629 non-null  float64
 7   jobs_1000     115629 non-null  float64
 8   loc_quotient  115629 non-null  float64
 9   h_mean        115629 non-null  float64
 10  a_mean        115629 non-null  float64
 11  mean_prse     115629 non-null  float64
 12  h_pct10       115629 non-null  float64
 13  h_pct25       115629 non-null  float64
 14  h_median      115629 non-null  float64
 15  h_pct75       115629 non-null  float64
 16  h_pct90       115629 non-null  float64
 17  a_pct10       115629 non-null  float64
 18  a_pc

In [21]:
bls_metro_2019_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118165 entries, 0 to 118164
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   area          118165 non-null  int64  
 1   citi_state    118165 non-null  object 
 2   occ_code      118165 non-null  object 
 3   occ_title     118165 non-null  object 
 4   tot_emp       118165 non-null  float64
 5   emp_prse      118165 non-null  float64
 6   jobs_1000     118165 non-null  float64
 7   loc_quotient  118165 non-null  float64
 8   h_mean        118165 non-null  float64
 9   a_mean        118165 non-null  float64
 10  mean_prse     118165 non-null  float64
 11  h_pct10       118165 non-null  float64
 12  h_pct25       118165 non-null  float64
 13  h_median      118165 non-null  float64
 14  h_pct75       118165 non-null  float64
 15  h_pct90       118165 non-null  float64
 16  a_pct10       118165 non-null  float64
 17  a_pct25       118165 non-null  float64
 18  a_me

In [22]:
bls_metro_2018_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118961 entries, 0 to 118960
Data columns (total 23 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   prim_state    118961 non-null  object 
 1   area          118961 non-null  int64  
 2   citi_state    118961 non-null  object 
 3   occ_code      118961 non-null  object 
 4   occ_title     118961 non-null  object 
 5   tot_emp       118961 non-null  float64
 6   emp_prse      118961 non-null  float64
 7   jobs_1000     118961 non-null  float64
 8   loc_quotient  118961 non-null  float64
 9   h_mean        118961 non-null  float64
 10  a_mean        118961 non-null  float64
 11  mean_prse     118961 non-null  float64
 12  h_pct10       118961 non-null  float64
 13  h_pct25       118961 non-null  float64
 14  h_median      118961 non-null  float64
 15  h_pct75       118961 non-null  float64
 16  h_pct90       118961 non-null  float64
 17  a_pct10       118961 non-null  float64
 18  a_pc

# match 3 dataframe and merge

In [23]:
#drop prim_state to match 2019
bls_metro_2018_df.drop(['prim_state'],axis=1, inplace=True)
bls_metro_2018_df.head(5)

,area,citi_state,occ_code,occ_title,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,Date
0,11260,"Anchorage, AK",00-0000,All Occupations,171770.0,1.1,1000.000,1.00,28.57,59430.0,2.1,11.41,14.94,22.68,35.82,51.21,23740.0,31070.0,47170.0,74510.0,106510.0,May 2018
1,11260,"Anchorage, AK",11-0000,Management Occupations,10610.0,2.5,61.779,1.17,55.59,115630.0,1.8,26.68,36.03,48.94,64.70,93.24,55490.0,74950.0,101800.0,134580.0,193930.0,May 2018
2,11260,"Anchorage, AK",11-2011,Advertising and Promotions Managers,50.0,16.5,0.282,1.62,45.98,95640.0,7.8,23.96,32.01,37.84,49.67,89.90,49830.0,66580.0,78710.0,103320.0,186990.0,May 2018
3,11260,"Anchorage, AK",11-2021,Marketing Managers,150.0,12.9,0.867,0.52,49.62,103220.0,3.4,28.73,35.57,45.62,59.58,77.63,59760.0,73980.0,94890.0,123930.0,161480.0,May 2018
4,11260,"Anchorage, AK",11-2031,Public Relations and Fundraising Managers,100.0,13.7,0.557,1.11,55.12,114650.0,8.4,28.48,35.40,48.55,65.02,78.77,59240.0,73640.0,100980.0,135240.0,163840.0,May 2018


In [24]:
bls_metro_2018_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118961 entries, 0 to 118960
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   area          118961 non-null  int64  
 1   citi_state    118961 non-null  object 
 2   occ_code      118961 non-null  object 
 3   occ_title     118961 non-null  object 
 4   tot_emp       118961 non-null  float64
 5   emp_prse      118961 non-null  float64
 6   jobs_1000     118961 non-null  float64
 7   loc_quotient  118961 non-null  float64
 8   h_mean        118961 non-null  float64
 9   a_mean        118961 non-null  float64
 10  mean_prse     118961 non-null  float64
 11  h_pct10       118961 non-null  float64
 12  h_pct25       118961 non-null  float64
 13  h_median      118961 non-null  float64
 14  h_pct75       118961 non-null  float64
 15  h_pct90       118961 non-null  float64
 16  a_pct10       118961 non-null  float64
 17  a_pct25       118961 non-null  float64
 18  a_me

In [25]:
bls_metro_2020_df.drop(['prim_state'],axis=1, inplace=True)
bls_metro_2020_df

,area,citi_state,occ_code,occ_title,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,Date
0,10180,"Abilene, TX",00-0000,All Occupations,66060.0,1.9,1000.000,1.00,20.64,42930.0,1.8,9.13,11.40,16.41,24.82,36.23,18990.0,23710.0,34130.0,51620.0,75370.0,May 2020
1,10180,"Abilene, TX",11-0000,Management Occupations,2910.0,4.5,44.068,0.77,42.87,89160.0,2.2,17.38,26.74,36.37,51.68,73.88,36150.0,55620.0,75640.0,107500.0,153670.0,May 2020
2,10180,"Abilene, TX",11-1021,General and Operations Managers,1320.0,7.4,20.036,1.19,40.38,83990.0,3.5,12.67,24.02,32.96,50.11,72.93,26350.0,49960.0,68550.0,104230.0,151700.0,May 2020
3,10180,"Abilene, TX",11-2030,Public Relations and Fundraising Managers,40.0,31.5,0.549,0.94,45.93,95540.0,20.7,28.71,32.78,36.24,41.23,83.81,59710.0,68180.0,75380.0,85760.0,174320.0,May 2020
4,10180,"Abilene, TX",11-3010,Administrative Services and Facilities Managers,140.0,12.3,2.045,0.92,37.79,78600.0,3.5,24.93,28.28,35.06,45.45,55.25,51850.0,58820.0,72920.0,94530.0,114920.0,May 2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115624,79600,"Worcester, MA-CT",53-7062,"Laborers and Freight, Stock, and Material Move...",4770.0,7.8,17.944,0.89,16.52,34360.0,1.7,12.87,13.43,14.91,18.41,22.96,26780.0,27930.0,31000.0,38290.0,47760.0,May 2020
115625,79600,"Worcester, MA-CT",53-7063,Machine Feeders and Offbearers,180.0,17.5,0.660,1.43,17.57,36540.0,5.1,13.35,14.31,16.11,20.42,24.29,27770.0,29770.0,33500.0,42470.0,50530.0,May 2020
115626,79600,"Worcester, MA-CT",53-7064,"Packers and Packagers, Hand",1220.0,21.6,4.592,1.07,15.68,32620.0,2.4,12.87,13.42,14.68,16.60,19.62,26780.0,27910.0,30540.0,34530.0,40800.0,May 2020
115627,79600,"Worcester, MA-CT",53-7065,Stockers and Order Fillers,4300.0,4.5,16.161,1.02,16.40,34100.0,1.2,12.89,13.65,14.99,18.13,22.65,26810.0,28400.0,31170.0,37700.0,47100.0,May 2020


In [26]:
bls_metro_2020_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115629 entries, 0 to 115628
Data columns (total 22 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   area          115629 non-null  int64  
 1   citi_state    115629 non-null  object 
 2   occ_code      115629 non-null  object 
 3   occ_title     115629 non-null  object 
 4   tot_emp       115629 non-null  float64
 5   emp_prse      115629 non-null  float64
 6   jobs_1000     115629 non-null  float64
 7   loc_quotient  115629 non-null  float64
 8   h_mean        115629 non-null  float64
 9   a_mean        115629 non-null  float64
 10  mean_prse     115629 non-null  float64
 11  h_pct10       115629 non-null  float64
 12  h_pct25       115629 non-null  float64
 13  h_median      115629 non-null  float64
 14  h_pct75       115629 non-null  float64
 15  h_pct90       115629 non-null  float64
 16  a_pct10       115629 non-null  float64
 17  a_pct25       115629 non-null  float64
 18  a_me

In [27]:
#check columns in all 3 df 
bls_metro_2020_df.columns.to_list()


['area',
 'citi_state',
 'occ_code',
 'occ_title',
 'tot_emp',
 'emp_prse',
 'jobs_1000',
 'loc_quotient',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90',
 'Date']

In [28]:
bls_metro_2019_df.columns.to_list()

['area',
 'citi_state',
 'occ_code',
 'occ_title',
 'tot_emp',
 'emp_prse',
 'jobs_1000',
 'loc_quotient',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90',
 'Date']

In [29]:
bls_metro_2018_df.columns.to_list()

['area',
 'citi_state',
 'occ_code',
 'occ_title',
 'tot_emp',
 'emp_prse',
 'jobs_1000',
 'loc_quotient',
 'h_mean',
 'a_mean',
 'mean_prse',
 'h_pct10',
 'h_pct25',
 'h_median',
 'h_pct75',
 'h_pct90',
 'a_pct10',
 'a_pct25',
 'a_median',
 'a_pct75',
 'a_pct90',
 'Date']

In [33]:
merged_1819_df = pd.concat([bls_metro_2018_df, bls_metro_2019_df])
merged_1819_df.head(10)

,area,citi_state,occ_code,occ_title,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,Date
0,11260,"Anchorage, AK",00-0000,All Occupations,171770.0,1.1,1000.000,1.00,28.57,59430.0,2.1,11.41,14.94,22.68,35.82,51.21,23740.0,31070.0,47170.0,74510.0,106510.0,May 2018
1,11260,"Anchorage, AK",11-0000,Management Occupations,10610.0,2.5,61.779,1.17,55.59,115630.0,1.8,26.68,36.03,48.94,64.70,93.24,55490.0,74950.0,101800.0,134580.0,193930.0,May 2018
2,11260,"Anchorage, AK",11-2011,Advertising and Promotions Managers,50.0,16.5,0.282,1.62,45.98,95640.0,7.8,23.96,32.01,37.84,49.67,89.90,49830.0,66580.0,78710.0,103320.0,186990.0,May 2018
3,11260,"Anchorage, AK",11-2021,Marketing Managers,150.0,12.9,0.867,0.52,49.62,103220.0,3.4,28.73,35.57,45.62,59.58,77.63,59760.0,73980.0,94890.0,123930.0,161480.0,May 2018
4,11260,"Anchorage, AK",11-2031,Public Relations and Fundraising Managers,100.0,13.7,0.557,1.11,55.12,114650.0,8.4,28.48,35.40,48.55,65.02,78.77,59240.0,73640.0,100980.0,135240.0,163840.0,May 2018
5,11260,"Anchorage, AK",11-3011,Administrative Services Managers,360.0,5.8,2.077,1.06,45.88,95430.0,3.6,28.30,34.62,41.47,49.85,68.03,58860.0,72010.0,86260.0,103690.0,141490.0,May 2018
6,11260,"Anchorage, AK",11-3021,Computer and Information Systems Managers,260.0,5.9,1.518,0.56,51.21,106510.0,2.0,35.17,41.95,50.21,60.50,70.74,73160.0,87260.0,104440.0,125830.0,147140.0,May 2018
7,11260,"Anchorage, AK",11-3031,Financial Managers,540.0,12.4,3.143,0.75,55.50,115430.0,2.9,31.45,39.14,50.99,62.96,85.13,65410.0,81410.0,106060.0,130960.0,177080.0,May 2018
8,11260,"Anchorage, AK",11-3051,Industrial Production Managers,70.0,10.7,0.406,0.32,52.72,109660.0,4.6,29.61,36.99,50.57,61.24,78.19,61590.0,76940.0,105180.0,127380.0,162630.0,May 2018
9,11260,"Anchorage, AK",11-3061,Purchasing Managers,50.0,11.4,0.294,0.61,51.93,108020.0,6.2,32.12,39.09,53.20,60.02,64.53,66810.0,81300.0,110650.0,124830.0,134220.0,May 2018


In [34]:
merged_181920_df = pd.concat([merged_1819_df, bls_metro_2020_df])
merged_181920_df.head(10)

,area,citi_state,occ_code,occ_title,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,Date
0,11260,"Anchorage, AK",00-0000,All Occupations,171770.0,1.1,1000.000,1.00,28.57,59430.0,2.1,11.41,14.94,22.68,35.82,51.21,23740.0,31070.0,47170.0,74510.0,106510.0,May 2018
1,11260,"Anchorage, AK",11-0000,Management Occupations,10610.0,2.5,61.779,1.17,55.59,115630.0,1.8,26.68,36.03,48.94,64.70,93.24,55490.0,74950.0,101800.0,134580.0,193930.0,May 2018
2,11260,"Anchorage, AK",11-2011,Advertising and Promotions Managers,50.0,16.5,0.282,1.62,45.98,95640.0,7.8,23.96,32.01,37.84,49.67,89.90,49830.0,66580.0,78710.0,103320.0,186990.0,May 2018
3,11260,"Anchorage, AK",11-2021,Marketing Managers,150.0,12.9,0.867,0.52,49.62,103220.0,3.4,28.73,35.57,45.62,59.58,77.63,59760.0,73980.0,94890.0,123930.0,161480.0,May 2018
4,11260,"Anchorage, AK",11-2031,Public Relations and Fundraising Managers,100.0,13.7,0.557,1.11,55.12,114650.0,8.4,28.48,35.40,48.55,65.02,78.77,59240.0,73640.0,100980.0,135240.0,163840.0,May 2018
5,11260,"Anchorage, AK",11-3011,Administrative Services Managers,360.0,5.8,2.077,1.06,45.88,95430.0,3.6,28.30,34.62,41.47,49.85,68.03,58860.0,72010.0,86260.0,103690.0,141490.0,May 2018
6,11260,"Anchorage, AK",11-3021,Computer and Information Systems Managers,260.0,5.9,1.518,0.56,51.21,106510.0,2.0,35.17,41.95,50.21,60.50,70.74,73160.0,87260.0,104440.0,125830.0,147140.0,May 2018
7,11260,"Anchorage, AK",11-3031,Financial Managers,540.0,12.4,3.143,0.75,55.50,115430.0,2.9,31.45,39.14,50.99,62.96,85.13,65410.0,81410.0,106060.0,130960.0,177080.0,May 2018
8,11260,"Anchorage, AK",11-3051,Industrial Production Managers,70.0,10.7,0.406,0.32,52.72,109660.0,4.6,29.61,36.99,50.57,61.24,78.19,61590.0,76940.0,105180.0,127380.0,162630.0,May 2018
9,11260,"Anchorage, AK",11-3061,Purchasing Managers,50.0,11.4,0.294,0.61,51.93,108020.0,6.2,32.12,39.09,53.20,60.02,64.53,66810.0,81300.0,110650.0,124830.0,134220.0,May 2018


In [35]:
merged_181920_df.tail(10)

,area,citi_state,occ_code,occ_title,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,Date
115619,79600,"Worcester, MA-CT",53-3099,"Motor Vehicle Operators, All Other",70.0,27.6,0.266,0.72,17.98,37390.0,7.6,11.77,13.28,17.38,22.43,25.31,24470.0,27620.0,36160.0,46640.0,52650.0,May 2020
115620,79600,"Worcester, MA-CT",53-6021,Parking Attendants,190.0,27.8,0.710,0.80,15.33,31900.0,2.3,13.03,13.70,14.82,16.72,18.96,27100.0,28500.0,30830.0,34780.0,39440.0,May 2020
115621,79600,"Worcester, MA-CT",53-6031,Automotive and Watercraft Service Attendants,40.0,22.6,0.138,0.17,14.50,30160.0,2.8,12.97,13.48,14.34,15.20,16.64,26970.0,28050.0,29830.0,31620.0,34620.0,May 2020
115622,79600,"Worcester, MA-CT",53-7051,Industrial Truck and Tractor Operators,1280.0,7.9,4.796,1.04,17.98,37400.0,2.9,13.74,15.09,17.54,20.14,23.84,28580.0,31380.0,36490.0,41890.0,49580.0,May 2020
115623,79600,"Worcester, MA-CT",53-7061,Cleaners of Vehicles and Equipment,280.0,15.8,1.056,0.43,16.73,34790.0,5.0,12.77,12.92,14.87,19.59,23.75,26550.0,26870.0,30920.0,40750.0,49390.0,May 2020
115624,79600,"Worcester, MA-CT",53-7062,"Laborers and Freight, Stock, and Material Move...",4770.0,7.8,17.944,0.89,16.52,34360.0,1.7,12.87,13.43,14.91,18.41,22.96,26780.0,27930.0,31000.0,38290.0,47760.0,May 2020
115625,79600,"Worcester, MA-CT",53-7063,Machine Feeders and Offbearers,180.0,17.5,0.660,1.43,17.57,36540.0,5.1,13.35,14.31,16.11,20.42,24.29,27770.0,29770.0,33500.0,42470.0,50530.0,May 2020
115626,79600,"Worcester, MA-CT",53-7064,"Packers and Packagers, Hand",1220.0,21.6,4.592,1.07,15.68,32620.0,2.4,12.87,13.42,14.68,16.60,19.62,26780.0,27910.0,30540.0,34530.0,40800.0,May 2020
115627,79600,"Worcester, MA-CT",53-7065,Stockers and Order Fillers,4300.0,4.5,16.161,1.02,16.40,34100.0,1.2,12.89,13.65,14.99,18.13,22.65,26810.0,28400.0,31170.0,37700.0,47100.0,May 2020
115628,79600,"Worcester, MA-CT",53-7081,Refuse and Recyclable Material Collectors,100.0,16.6,0.388,0.45,22.05,45870.0,4.9,13.41,15.67,22.92,27.45,30.28,27900.0,32580.0,47680.0,57090.0,62980.0,May 2020


In [39]:
#spliting citi_state col
merged_181920_df[['city','state']] = merged_181920_df.citi_state.str.split(",",expand=True)
merged_181920_df.head(5)

,area,citi_state,occ_code,occ_title,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,Date,city,state
0,11260,"Anchorage, AK",00-0000,All Occupations,171770.0,1.1,1000.000,1.00,28.57,59430.0,2.1,11.41,14.94,22.68,35.82,51.21,23740.0,31070.0,47170.0,74510.0,106510.0,May 2018,Anchorage,AK
1,11260,"Anchorage, AK",11-0000,Management Occupations,10610.0,2.5,61.779,1.17,55.59,115630.0,1.8,26.68,36.03,48.94,64.70,93.24,55490.0,74950.0,101800.0,134580.0,193930.0,May 2018,Anchorage,AK
2,11260,"Anchorage, AK",11-2011,Advertising and Promotions Managers,50.0,16.5,0.282,1.62,45.98,95640.0,7.8,23.96,32.01,37.84,49.67,89.90,49830.0,66580.0,78710.0,103320.0,186990.0,May 2018,Anchorage,AK
3,11260,"Anchorage, AK",11-2021,Marketing Managers,150.0,12.9,0.867,0.52,49.62,103220.0,3.4,28.73,35.57,45.62,59.58,77.63,59760.0,73980.0,94890.0,123930.0,161480.0,May 2018,Anchorage,AK
4,11260,"Anchorage, AK",11-2031,Public Relations and Fundraising Managers,100.0,13.7,0.557,1.11,55.12,114650.0,8.4,28.48,35.40,48.55,65.02,78.77,59240.0,73640.0,100980.0,135240.0,163840.0,May 2018,Anchorage,AK


In [40]:
merged_181920_df.tail(5)

,area,citi_state,occ_code,occ_title,tot_emp,emp_prse,jobs_1000,loc_quotient,h_mean,a_mean,mean_prse,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90,Date,city,state
115624,79600,"Worcester, MA-CT",53-7062,"Laborers and Freight, Stock, and Material Move...",4770.0,7.8,17.944,0.89,16.52,34360.0,1.7,12.87,13.43,14.91,18.41,22.96,26780.0,27930.0,31000.0,38290.0,47760.0,May 2020,Worcester,MA-CT
115625,79600,"Worcester, MA-CT",53-7063,Machine Feeders and Offbearers,180.0,17.5,0.660,1.43,17.57,36540.0,5.1,13.35,14.31,16.11,20.42,24.29,27770.0,29770.0,33500.0,42470.0,50530.0,May 2020,Worcester,MA-CT
115626,79600,"Worcester, MA-CT",53-7064,"Packers and Packagers, Hand",1220.0,21.6,4.592,1.07,15.68,32620.0,2.4,12.87,13.42,14.68,16.60,19.62,26780.0,27910.0,30540.0,34530.0,40800.0,May 2020,Worcester,MA-CT
115627,79600,"Worcester, MA-CT",53-7065,Stockers and Order Fillers,4300.0,4.5,16.161,1.02,16.40,34100.0,1.2,12.89,13.65,14.99,18.13,22.65,26810.0,28400.0,31170.0,37700.0,47100.0,May 2020,Worcester,MA-CT
115628,79600,"Worcester, MA-CT",53-7081,Refuse and Recyclable Material Collectors,100.0,16.6,0.388,0.45,22.05,45870.0,4.9,13.41,15.67,22.92,27.45,30.28,27900.0,32580.0,47680.0,57090.0,62980.0,May 2020,Worcester,MA-CT


In [37]:
merged_181920_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 352755 entries, 0 to 115628
Data columns (total 24 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   area          352755 non-null  int64  
 1   citi_state    352755 non-null  object 
 2   occ_code      352755 non-null  object 
 3   occ_title     352755 non-null  object 
 4   tot_emp       352755 non-null  float64
 5   emp_prse      352755 non-null  float64
 6   jobs_1000     352755 non-null  float64
 7   loc_quotient  352755 non-null  float64
 8   h_mean        352755 non-null  float64
 9   a_mean        352755 non-null  float64
 10  mean_prse     352755 non-null  float64
 11  h_pct10       352755 non-null  float64
 12  h_pct25       352755 non-null  float64
 13  h_median      352755 non-null  float64
 14  h_pct75       352755 non-null  float64
 15  h_pct90       352755 non-null  float64
 16  a_pct10       352755 non-null  float64
 17  a_pct25       352755 non-null  float64
 18  a_me

In [41]:
merged_181920_df.to_csv('data_files/merged_metrodata_181920.csv')

In [31]:
#make a list of cities 
# citi_state_metro = merged_181920_df["citi_state"].unique()
# citi_state_metro 